#Bases de datos
Finales de los 70s, necesidad de almacenar y analizar secuencias.

80s -> Amos Bairoch, basandose en Atlas de Dayhoft cambia el formato a computacional
-> lanzamiento de **Swiss-Prot** (1986)

##Swiss-Prot
* resultados experimentales
* necesidad de biocuradores
* expansión de la base de datos

90s escasez de financiamiento

1998 -> creación del SIB

## UniProt (2002)
Consorcio:
* Swiss-Prot (anotación manual)
* TrEMBL (anotación automática)
* PIR protein database

*Mantenimiento y anotación constante

### PDB vs UniProt

PDB: repositorio de estructuras 3D

UniProt: Base de datos de secuencias y anotación de proteínas

## ExPASy (1993)
Forma de interactuar con todas las bases de datos (160) más herramientas.
Los recursos se alojan en diferentes grupos.

###Prosite
Base de datos de familias o dominios de proteínas

ProRULE: genera distintas anotaciones en formato swiss-prot automaticamente

In [ ]:
# Para UniProt
SwissProt.read()
SwissProt.parse)
# ExPASy (obtiene archivo swiss prot)
ExPASy.get_sprot_raw()

In [ ]:
from Bio import Entrez, SeqIO
Entrez.email = "mmayen@lcg.unam.mx"

#Busqueda del organismo y gen en la base de datos de proteínas
handle = Entrez.esearch(db="protein", term="Aedes aegypti[Orgn] AND APY[Gene]") 
record = Entrez.read(handle) 
record["Count"]

# Extraer desde la base de datos de protein
handle = Entrez.efetch(db="protein", id="193806340", rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
handle.close()
record.id
record.annotations['db_source']

# Información en otras bases de datos como UniProt
db_source = record.annotations['db_source']
apy_prot = record.annotations['accessions']

# O a partir de la base de datos de gene
handle = Entrez.esearch(db="gene", term="Aedes aegypti[Orgn] AND apy[Gene]")
record = Entrez.read(handle)   

handle = Entrez.efetch(db="gene", id=record['IdList'][0], retmode="xml")
record = Entrez.read(handle, "genbank")
record[0]['Entrezgene_gene']['Gene-ref']['Gene-ref_syn']


## Ejercicio 1
Buscar el accession de la base de datos de UniProtKB del gen DEFA del mosquito Aedes aegypti

In [ ]:
from Bio import Entrez, SeqIo
Entrez.email = "mmayen@lcg.unam.mx"

handle = Entrez.esearch(db="protein", term="Aedes aegypti[Orgn] AND DEFA[Gene]")
record = Entrez.read(handle)

record["count"]
print(record["IdList"])

handle = Entrez.efetch(db="protein", id=record["IdList"][0], rettype="gb", retmode="text")
record = SeqIo.read(handle, "genbank")

print(record.annotations["db_source"])
DEFA_prot = record.annotations["accessions"]
db_source = record.annotations["db_source"]


In [ ]:
from Bio import ExPASy 
from Bio import SwissProt

# ExPASy.get_sprot_raw() permite obtener un archivo SwissProt crudo
handle = ExPASy.get_sprot_raw(apy_prot[0])
# SwissProt.read() o SwissProt.parse() permite leer un archivo SwissProt crudo y lo convierte en un objeto
record = SwissProt.read(handle)
record.__dict__.keys()

## Ejercicio 2
Con el accession de la base de datos de UniProtKB del gen DEFA del mosquito Aedes aegypti , obtener el archivo de UniProt e imprimir:
* Fecha de creación
* Cuándo actualizaron su anotación
* ID de taxonomía

In [ ]:
handle = ExPASy.get_sprot_raw('P91793')
record = SwissProt.read(handle)

#Creación, actualización, Id taxonomía
print(record.created)
print(record.annotation_update)
print(record.taxonomy_id)

In [ ]:
# Desde un archivo descargado con un solo record
handle = open("./files/clase_4/O23729.txt")
record = SwissProt.read(handle)

record.__dict__.keys()
record.gene_name
record.sequence

# Multiples record
handle = open("./files/clase_4/mini_uniprot.dat")
descriptions = [record.description for record in SwissProt.parse(handle)]
len(descriptions)

In [ ]:
### Crear objeto SeqRecord ###

import Bio.SeqRecord, Bio.Seq

seqRec = Bio.SeqRecord.SeqRecord(seq=Bio.Seq.Seq(record.sequence), id=record.entry_name, name=record.organism, description=record.description)
pprint(seqRec.__dict__)

# Podemos trabajar con un objeto SeqRecord
seqRec.format('fasta')

# Tambén pudemos utilizar Bio.SeqUtils
from Bio.SeqUtils import seq3, seq1, molecular_weight
prot = seqRec.seq
# Convertir a código de 3 letras
seq3(prot)[0:12]
# Convertir a código de 1 letra
seq1( seq3(prot)[0:12] )
# Peso molecular
molecular_weight(prot, seq_type='protein')


## Ejercicio 3
Con el archivo de UniProtKB del gen DEFA del mosquito Aedes aegypti, crear un objeto SeqRecord

In [ ]:
 import Bio.SeqRecord, Bio.Seq

 objeto_SeqRecord = Bio.SeqRecord.SeqRecord(seq=Bio.Seq.Seq(record.seq),id=record.entry_name,name=record.organism,description=record.description)
 
 for key, item in objeto_SeqRecord.__dict__.items():
  print(key, item)

print(objeto_SeqRecord.format('fasta'))


## prosite
¿Hay información de prosite en el archivo SwissProt?

In [ ]:
for reference in record.cross_references:
    if 'PROSITE' in reference:
        print(reference)

In [ ]:
# Usando prosite (read)
from Bio import ExPASy
from Bio.ExPASy import Prosite

handle = ExPASy.get_prosite_raw("PS00785")
record = Prosite.read(handle)

print(record.name)
print(record.type)  #PATTERN/MATRIX/RULE
print(record.pdoc)  #Documento asociado a ese patrón


In [ ]:
# Usando prosite (prodoc)
from Bio import ExPASy
from Bio.ExPASy import Prodoc

handle = ExPASy.get_prosite_raw(record.pdoc)
record = Prodoc.read(handle)
print(record.text)

### Scan prosite
Permite escanear proteínas en busca de coincidencias con la colección de motivos PROSITE.

Puede ser con una secuencia, un ID de PDB, un ID de UniProt, etc

In [ ]:
#Buscar motivos ya anotados desde una secuencia
sequence = "MEHKEVVLLLLLFLKSGQGEPLDDYVNTQGASLFSVTKKQLGAGSIEECAAKCEEDEEFTCRAFQYHSKEQQCVIMAENRKSSIIIRMRDVVLFEKKVYLSECKTGNGKNYRGTMSKTKN"

from Bio.ExPASy import ScanProsite
handle = ScanProsite.scan(seq=sequence)
result = ScanProsite.read(handle)

print(type(result))
print(result[0])
# 'Signature_ac'="PS50948"


from Bio import ExPASy
from Bio.ExPASy import Prodoc

handle = ExPASy.get_prosite_raw("PS50948") 
record = Prosite.read(handle)
print (record.name)

handle = ExPASy.get_prosite_raw(record.pdoc)
record = Prodoc.read(handle)